# Timeseries Forecasting of Solar Flares using LSTM

In [ ]:
import pandas as pd
import pickle
from keras.models import load_model
from keras import Sequential
from keras.layers import LSTM, SimpleRNN, GRU, Dense
from keras.losses import SparseCategoricalCrossentropy
from keras.callbacks import EarlyStopping
import numpy as np
from sklearn.metrics import ConfusionMatrixDisplay
import os
from sklearn.metrics import precision_recall_fscore_support

In [ ]:
def loadPickledFile(fileName):
    bytes_in = bytearray(0)
    max_bytes = 2**31 - 1
    input_size = os.path.getsize(fileName)
    with open(fileName, 'rb') as file:
        for _ in range(0, input_size, max_bytes):
            bytes_in += file.read(max_bytes)
        obj = pickle.loads(bytes_in)
        file.close()
    return obj

## Data Cleaning

As shown in analysisAndVisualization, there are 6 magnetic field parameters which are highly correlated with flare type. These 6 are the features we will use for the LSTM. The LSTM model will take in 24 hours of past magnetic field data, and predict what the most severe solar flare in the next hour will be. The data contains a major class imbalance, so we created a new dataset with more balanced classes in order to prevent overfitting. We will train on the balanced data, then test on the real data. The code for creation of this balanced dataset can be found in `create_balanced_datasets.py`. 

In [ ]:
X_train = loadPickledFile("balanced_data/X_train.pck")
X_test = loadPickledFile("balanced_data/X_test.pck")
y_train = loadPickledFile("balanced_data/y_train.pck")
y_test = loadPickledFile("balanced_data/y_test.pck")

## Create Neural Net

Recurrent Neural Networks such as the LSTM are well-suited for timeseries forecasting, so our model will just be a simple LSTM. This is a multi-class classification problem, so categorical cross entropy is the most suitable loss function, and the softmax function is the most suitable activation function for the output layer.

In [ ]:
num_features = X_train[0].shape[1]
num_classes = 5
model = Sequential()
# Input Layer
model.add(Dense(num_features, input_shape=X_train[0].shape,activation='relu'))
# Hidden Layers: RNN
model.add(LSTM(units=20*num_features)) # Add return_sequences=True if you want to add more architecture
# model.add(GRU(units=2*num_features, return_sequences=True))
# model.add(SimpleRNN(units=2*num_features))
# Output Layer
model.add(Dense(num_classes, activation='softmax'))

model.compile(optimizer='adam', loss=SparseCategoricalCrossentropy())
model.summary()

## Training

We will reserve 20% of the training data for validation, and train the network. If the validation loss stops improving, we will stop training so the model does not overfit. 

In [ ]:
epochs = 100
val_split = 0.2
model.fit(X_train,
          y_train,
          validation_split=val_split,
          epochs=epochs,
          verbose=1,
          callbacks=[EarlyStopping(monitor='val_loss', patience=5)],
          shuffle=True)

## Persisting Model

Next, we persist the model so it can be used later.

In [ ]:
modelName = "lstm-balanced"
model.save(f"persisted_nets/{modelName}")

## Loading and Evaluating Model

Now, we can load the model back in from the file, and evaluate it. First, we will evaluate it on the balanced dataset. Then, we can evaluate it on real timeseries data. 

In [ ]:
modelName = "lstm-balanced"
model = load_model(f"persisted_nets/{modelName}")

In [ ]:
classLabels = {
    0: "0",
    1: "B",
    2: "C",
    3: "M",
    4: "X"
}

def threshold_output(output):
    thresholded = []
    for arr in output:
        thresholded.append(np.array([np.array([np.argmax(arr)])]))
    return np.array(thresholded)
    
def printMetrics(p,r,f,s):
    for i in range(len(p)):
        print(f"Metrics for {classLabels[i]} class flares")
        print(f"\tPrecision: {p[i]}")
        print(f"\tRecall:    {r[i]}")
        print(f"\tF-Score:   {f[i]}")
        print(f"\tSupport:   {s[i]}")

def evaluate_model(model,X_test,y_test, plotTitle):
    y_pred = model.predict(X_test)
    y_pred = threshold_output(y_pred)

    p,r,f,s = precision_recall_fscore_support(y_test.flatten(), y_pred.flatten())

    printMetrics(p,r,f,s)
    
    disp = ConfusionMatrixDisplay.from_predictions(y_test.flatten(), y_pred.flatten(), display_labels=["No Flare","B", "C", "M", "X"])
    disp.ax_.set_title(plotTitle)

In [ ]:
evaluate_model(model, X_test, y_test, "Confusion Matrix for Balanced Dataset")

As shown in the metrics and confusion matrix above, the model has a difficult time predicting X-class flares. This is likely because the number of X-class flares is small. In the future, we can improve the prediction of X-class flares by using a more deep, sophisticated model.

## Evaluation of the Model on Unbalanced Data

Real solar flare data is **very** unbalanced, so we need to see how our model performs on the real, unbalanced data. The data used for this evaluation is just data from the year 2014, transformed into a format which the LSTM can take. Creation of this dataset is done in `create_LSTM_datasets.py`

In [ ]:
X_train = loadPickledFile("timeseries/X_train.pck")
X_test = loadPickledFile("timeseries/X_test.pck")
y_train = loadPickledFile("timeseries/y_train.pck")
y_test = loadPickledFile("timeseries/y_test.pck")

In [ ]:
evaluate_model(model, X_test, y_test, "Confusion Matrix for unbalanced 2014 time series data")

As shown in the confusion matrix and metrics above, our model has difficulties discerning between the "No Flare" case and B- and C- class flares. Because of the severity of M and X class flares, they are more important to predict than the less severe solar flares, and we see that the recall of M-class flares is around 40%, which for a simple model is not bad for timeseries prediction. The X-class prediction, however, is not at all good, with no flares being correctly predicted. Once again, a more sophisticated, deep machine learning model would help to make this better. With solar flares, predicting the "No Flare" condition is not as important as predicting a solar flare, so the metrics we care most about are the **recall** for each of the solar flare classes. The recall of the B- and C- class flares are around 60%, which is acceptable. The M- and X- class recall are not as favorable, as mentioned above.